In [ ]:
import requests
import json
import os

def collect_jira_issues(projects, date_start, date_end, max_results=100, auth=None, output_dir="jira_exports"):
    """
    Récupère les tickets JIRA via API JQL en plusieurs pages.

    projects : liste de clés projet JIRA (ex : ["ULIS", "SCM"])
    date_start : date de début (format YYYY-MM-DD)
    date_end : date de fin (format YYYY-MM-DD)
    max_results : nombre de résultats par page (max 1000 selon Jira)
    auth : tuple (username, token) pour authentification basique
    output_dir : dossier de sortie pour les JSON
    """

    if auth is None:
        raise ValueError("auth=(username, token) est requis")

    os.makedirs(output_dir, exist_ok=True)

    for project in projects:
        print(f"🔍 Récupération pour projet : {project}")
        start_at = 0
        index = 0

        while True:
            jql = f"created >= {date_start} AND created <= {date_end} AND project={project}"
            url = f"https://arvato-scs.atlassian.net/rest/api/latest/search"
            params = {
                "jql": jql,
                "startAt": start_at,
                "maxResults": max_results
            }

            response = requests.get(url, params=params, auth=auth)
            if response.status_code != 200:
                print(f"⚠️ Erreur {response.status_code} pour {project} : {response.text}")
                break

            data = response.json()
            issues = data.get("issues", [])
            total = data.get("total", 0)

            if not issues:
                print(f"✅ Fin (aucun ticket) à startAt={start_at}")
                break

            # Sauvegarde des résultats dans un fichier
            filename = os.path.join(output_dir, f"{project}_{index:04d}.json")
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2, ensure_ascii=False)

            print(f"💾 Fichier sauvegardé : {filename} ({len(issues)} tickets)")

            start_at += max_results
            index += 1

            if start_at >= total:
                print(f"✅ Tous les tickets récupérés pour {project} ({total} au total).")
                break


In [ ]:
collect_jira_issues(
    projects=["ULIS", "SCMF"],
    date_start="2017-01-01",
    date_end="2025-07-08",
    max_results=100,
    auth=("ton.email@entreprise.com", "ton_token_api")
)
